In [1]:
import os
from pathlib import Path

NOTEBOOK_PATH: Path = Path(os.path.abspath(''))

# print(f'当前运行在：{NOTEBOOK_PATH} 目录下。')

PATH_FOR_RESULT: Path = NOTEBOOK_PATH.joinpath('RESULT')

if not PATH_FOR_RESULT.exists():
    PATH_FOR_RESULT.mkdir()

# 完整的流程

我们把前面所有的步骤拼装起来：

1. 下载单个历史数据文件 <f>；

2. 解压缩 <f>（可以随即删除 <f>），得到一批数据文件 <d>；

3. 挨个读取数据文件 <d>，提取数据并写入数据库。
    
选择起始时间，按照年和月循环步骤 1 ~ 步骤 3。

代码如下：

In [2]:
# 本单元格涉及下载、读取，都是中金所特有行为。

from typing import Dict, List, Tuple, Any
from pathlib import Path
import datetime as dt

import requests


def download_cffex_history_data(save_path: Path, year: int, month: int) -> None:
    """
    下载中国金融期货交易所（中金所，CFFEX）的历史数据。
    :param save_path: 保存的位置。
    :param year: 需要下载数据的年份。
    :param month: 需要下载数据的月份。
    :return: None.
    """
    
    # 中金所历史数据 url 模板。
    url: str = 'http://www.cffex.com.cn/sj/historysj/{year:4d}{month:02d}/zip/{year:4d}{month:02d}.zip'

    # 中金所历史数据从 2010 年 4 月（股指期货）开始提供。
    start_year: int = 2010
    start_month: int = 4
    today: dt.date = dt.date.today()

    # 如果参数 <year> 和 <month> 不在合理范围，引发异常。
    if month < 1 or month > 12:
        raise ValueError(f'参数 <month> 取值范围在 [1, 12]。')
    if year < start_year or (year == start_year and month < start_month):
        raise ValueError(f'中金所历史数据自{start_year:4d}年{start_month:02d}月起开始提供。')
    if year > today.year or (year == today.year and month > today.month):
        raise ValueError(f'{year:4d}年{month:02d}月是未来日期。')

    # 如果参数 <save_path> 不存在，引发异常。
    if not save_path.exists():
        raise FileNotFoundError(f'目录 {save_path} 不存在。')

    # 下载。
    response = requests.get(url.format(year=year, month=month))
    
    # 如果下载不顺利，引发异常。
    if response.status_code != 200:
        raise requests.exceptions.HTTPError(
            f'下载 <{url.format(year=year, month=month)}> 时发生错误。'
        )

    # 保存文件。
    with open(save_path.joinpath(f'CFFEX_{year:4d}-{month:02d}.zip'), 'wb') as f:
        f.write(response.content)


def read_cffex_data(data_file: Path) -> Tuple[List[Dict[str, Any]]]:
    """
    读取中国金融期货交易所（中金所，CFFEX）的历史交易数据 (csv 文件)。
    :param data_file: Path，待读取的文件。
    :return: Tuple，共两项，每一项都是一个列表。
             前一个是期货数据，后一个是期权数据。
             每一个列表都是一个字典，字典的 key 是 str，为 key 的字典。
    """
    result_futures: List[Dict[str, Any]] = []
    result_option: List[Dict[str, Any]] = []
    
    # 从文件名中获得日期。
    filename: str = data_file.name[:8]
    date: dt.date = dt.date(
        year=int(filename[:4]),
        month=int(filename[4:6]),
        day=int(filename[6:8])
    )
    
    # 打开 <data_file> 读取数据。
    with open(data_file, mode='r', encoding='gbk') as csv_file:
        reader = csv.DictReader(csv_file)
        
        # 按行循环读取。
        for row in reader:
            # 忽略 <合约代码> 列为 <小计>、<合计> 的行。
            if row['合约代码'] == '小计' or row['合约代码'] == '合计':
                continue
            
            # 捕捉异常，打印出错的行，以便在出现我们未考虑的情况时，提供调试信息用于修改。
            try:
                # 合约代码，去除两端的空白（空格）
                symbol = row['合约代码'].strip()
                
                # 合约代码长度不超过6，期货
                if len(symbol) <= 6:
                    result_futures.append(
                        {
                            'exchange': 'CFFEX',
                            'date': date,
                            'symbol': symbol,
                            'open': float(row['今开盘']) if len(row['今开盘']) > 0 else 0.0,
                            'high': float(row['最高价']) if len(row['最高价']) > 0 else 0.0,
                            'low': float(row['最低价']) if len(row['最低价']) > 0 else 0.0,
                            'close': float(row['今收盘']) if len(row['今收盘']) > 0 else 0.0,
                            'settlement': float(row['今结算']),
                            'previous_settlement': float(row['前结算']),
                            'volume': int(row['成交量']) if len(row['成交量']) > 0 else 0,
                            'amount': float(row['成交金额']) if len(row['成交金额']) > 0 else 0.0,
                            'open_interest': int(float(row['持仓量'])),
                            'change_on_close': float(row['涨跌1']),
                            'change_on_settlement': float(row['涨跌2']),
                            'change_on_open_interest': int(float(row['持仓变化'])),
                        }
                    )
                # 合约代码长度超过6，期权
                else:
                    result_option.append(
                        {
                            'exchange': 'CFFEX',
                            'date': date,
                            'symbol': symbol,
                            'open': float(row['今开盘']) if len(row['今开盘']) > 0 else 0.0,
                            'high': float(row['最高价']) if len(row['最高价']) > 0 else 0.0,
                            'low': float(row['最低价']) if len(row['最低价']) > 0 else 0.0,
                            'close': float(row['今收盘']) if len(row['今收盘']) > 0 else 0.0,
                            'settlement': float(row['今结算']),
                            'previous_settlement': float(row['前结算']),
                            'volume': int(row['成交量']) if len(row['成交量']) > 0 else 0,
                            'amount': float(row['成交金额']) if len(row['成交金额']) > 0 else 0.0,
                            'open_interest': int(float(row['持仓量'])),
                            'change_on_close': float(row['涨跌1']),
                            'change_on_settlement': float(row['涨跌2']),
                            'change_on_open_interest': int(float(row['持仓变化'])),
                            'delta': float(row['Delta']),
                        }
                    )
            except ValueError as e:
                print(f'读取文件 {csv_file} 时发生错误。发生错误的行内容为：\n\t{row}')
    
    return result_futures, result_option

In [3]:
# 本单元格涉及解压，应该是通用的。

import zipfile


def unzip_file(zipped_file: Path, unzip_path: Path, keep: bool = False) -> List[Path]:
    """
    解压单个 zip 文件。
    Unzip a zip file to ten temporary directory defined in <CONFIGS>, and return the unzipped file path.
    :param zipped_file:   Path，待解压文件的路径。
    :param unzip_path: Path，保存被解压出来的文件的路径。
    :param keep:       bool，保留待解压文件的话，取值 True，否则解压完成后删除待解压文件。
    :return: list，被解压出来的文件的路径（Path）的列表。
    """
    
    # 如果参数 <zip_file> 不存在，引发异常。
    if not zipped_file.exists():
        raise FileNotFoundError(f'<{zipped_file}> 不存在。')

    # 如果参数 <unzip_path> 不存在，引发异常。
    if not unzip_path.exists():
        raise FileNotFoundError(f'<{unzip_path}> 不存在。')

    # 用 zipfile 模块的 ZipFile 打开待解压文件。
    zip_file = zipfile.ZipFile(zipped_file, 'r')
    
    # 生成解压文件列表
    result: List[Path] = [unzip_path.joinpath(filename) for filename in zip_file.namelist()]
    
    # 解压文件。
    zip_file.extractall(unzip_path)
    
    # 删除待解压文件
    if not keep:
        zipped_file.unlink()
    
    # 返回解压出来的文件列表
    return result

In [4]:
# 本单元格涉及数据库，不局限某一交易所。

from peewee import (
    SqliteDatabase,
    Model,
    AutoField,
    CharField,
    FixedCharField,
    DateField,
    FloatField,
    IntegerField,
)

class ModelBase(Model):
    """
    ORM 基对象。
    """
    class Meta:
        database = db
        legacy_table_names = False


# 定义期货数据（日线）。
class FuturesQuoteDaily(ModelBase):
    """
    期货交易数据（日线）。
    """
    
    # id 可以理解为一张表的行号。
    id = AutoField(primary_key=True)
    
    symbol = FixedCharField(verbose_name='代码', max_length=6)
    date = DateField(verbose_name='日期')
    open = FloatField(verbose_name='开盘价', null=True)
    high = FloatField(verbose_name='最高价', null=True)
    low = FloatField(verbose_name='最低价', null=True)
    close = FloatField(verbose_name='收盘价')
    volume = IntegerField(verbose_name='成交量')
    amount = FloatField(verbose_name='成交额')
    settlement = FloatField(verbose_name='结算价')
    open_interest = IntegerField(verbose_name='持仓量')
    
    def __str__(self) -> str:
        """
        为 print 提供数据。
        """
        return f'<FuturesQuoteDaily(' \
               f'symbol={self.symbol}, ' \
               f'date={self.date}, ' \
               f'open={self.open}, ' \
               f'high={self.high}, ' \
               f'low={self.low}, ' \
               f'close={self.close}, ' \
               f'volume={self.volume}, ' \
               f'amount={self.amount}, ' \
               f'settlement={self.settlement}, ' \
               f'open_interest={self.open_interest}' \
               f')>'


# 定义期权数据（日线）。
class OptionQuoteDaily(ModelBase):
    """
    期权交易数据（日线）。
    """
    
    id = AutoField(primary_key=True)
    
    symbol = FixedCharField(verbose_name='代码', max_length=6)
    date = DateField(verbose_name='日期')
    open = FloatField(verbose_name='开盘价', null=True)
    high = FloatField(verbose_name='最高价', null=True)
    low = FloatField(verbose_name='最低价', null=True)
    close = FloatField(verbose_name='收盘价')
    volume = IntegerField(verbose_name='成交量')
    amount = FloatField(verbose_name='成交额')
    settlement = FloatField(verbose_name='结算价')
    open_interest = IntegerField(verbose_name='持仓量')
    delta = FloatField(verbose_name='delta')
    
    def __str__(self) -> str:
        """
        为 print 提供数据。
        """
        return f'<OptionQuoteDaily(' \
               f'symbol={self.symbol}, ' \
               f'date={self.date}, ' \
               f'open={self.open}, ' \
               f'high={self.high}, ' \
               f'low={self.low}, ' \
               f'close={self.close}, ' \
               f'volume={self.volume}, ' \
               f'amount={self.amount}, ' \
               f'settlement={self.settlement}, ' \
               f'open_interest={self.open_interest}, ' \
               f'delta={self.delta}' \
               f')>'

In [5]:
# 总控

def collect_cffex_history_data(save_path: Path, db: ) -> None:
    """
    收集中国金融期货交易所（中金所，CFFEX）的全部历史数据。
    :param save_path: 保存的位置。
    :return: None.
    """
    start_year: int = 2010
    start_month: int = 4
    today: dt.date = dt.date.today()
    
    for year in range(start_year, today.year + 1):
        for month in range(1, 12 + 1):
            if year == 2010 and month < start_month:
                continue
            if year == today.year and month > today.month:
                break
            download_cffex_history_data(save_path=save_path, year=year, month=month)

SyntaxError: invalid syntax (Temp/ipykernel_15164/3944749791.py, line 3)